In [3]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import json
import torch
import pickle
import configparser
from ast import literal_eval
from hiko.dask import daskify
from sklearn.metrics import accuracy_score, roc_auc_score
from torch import nn

# Read in config
config = configparser.ConfigParser()
config.read('../ml_models_for_airflow/dbs3_config.ini')

db_engine = create_engine(config['AIRFLOW']['postgres_conn'])

pairs_mapping = literal_eval(config['MODEL']['pairs_mapping'])
pairs = tuple(pairs_mapping.values())

In [4]:
NUM_EPOCHS = 20

In [5]:
from training_data_loader import Dataset
from torch.utils import data

In [7]:
from transformer import Transformer

In [12]:
full_data_set = Dataset(config_location='../ml_models_for_airflow/dbs3_config.ini',
                      pairs=pairs,
                      seq_lenght=16,
                      num_features=3)

In [13]:
train_set_size = int(len(full_data_set)*0.85)
test_set_size = len(full_data_set) - train_set_size

In [14]:
trainset, valset = data.random_split(full_data_set,
                                     [train_set_size, test_set_size]
                                    )

nn.LogSoftmax + nn.NLLLoss -> to get probabilities, call `torch.exp` on the output

In [15]:
train_generator = data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2)
test_generator = data.DataLoader(valset, batch_size=256, shuffle=True, num_workers=2)

In [16]:
transformer = Transformer(emb=16,
                          heads=4,
                          depth=15,
                          num_classes=2,
                          num_features=3,
                          )

In [18]:
optimizer = torch.optim.Adam(transformer.parameters())

In [19]:
criterion = torch.nn.NLLLoss()
track_epoch_loss = []
train_auc = []
test_auc = []

In [20]:
for ep in range(NUM_EPOCHS):
    transformer.train()
    epoch_loss = 0
    temp_train_auc = 0
    
    for train_x, train_y in train_generator:
        
        predictions = transformer(train_x)
        loss = criterion(predictions, train_y)
        epoch_loss += loss.item()
        temp_train_auc += roc_auc_score(
            train_y.numpy(), torch.exp(predictions)[:, 1].detach().numpy())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_auc.append(temp_train_auc/len(train_generator))
    
    with torch.no_grad():
        transformer.eval()
        temp_test_auc = 0
        for test_x, test_y in test_generator:
            predictions = transformer(test_x)
            temp_test_auc += roc_auc_score(
                test_y.numpy(), torch.exp(predictions)[:, 1].numpy())

    test_auc.append(temp_test_auc/len(test_generator))

In [22]:
test_auc

[0.5729353295300637,
 0.5311213405315087,
 0.5393249237158102,
 0.5527567793002857,
 0.5686866319387328,
 0.5078949093791763,
 0.5138536078742745,
 0.559761183018957,
 0.5663652857967711,
 0.5226827670239378,
 0.49532188938134336,
 0.5753014656307325,
 0.5598566189462908,
 0.6023852937244736,
 0.546432449080692,
 0.5537755675388929,
 0.553190345581519,
 0.5539563941952396,
 0.5090521893967899,
 0.5415658797805475]